## Facial Recognition Training Pipeline Example
This notebook takes a directory of training images for a number of different people and trains a classifier to do facial recognition. The method is as follows:
    1. Crop and format input images
    2. Import model
    3. Perform transfer learning using fast.ai
    4. Output in ONNX format for use
    
Additionally, in order to load training images from their original directory, it must be structured as below:

<code>raw_training_data_dir <br>│<br>└───person_1<br>│ │───IMG1<br>│ │───IMG2<br>│ │ ....<br>└───person_2<br>| │───IMG1<br>| │───IMG2<br>| | ....</code>

In [ ]:
from fastai import *
from fastai.vision import *
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset
import torch.utils.model_zoo as model_zoo
import torch.onnx
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import cv2, dlib
import os, shutil, io, random, subprocess
os.system('pip install facenet_pytorch')
from facenet_pytorch import MTCNN, InceptionResnetV1
from datetime import datetime

bs=8
torch.cuda.is_available()
torch.cuda.set_device(0)

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

#### Functions utilizing OpenCV for image loading and manipulation

In [ ]:
# function to create list of cv2-loaded images in a given directory
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename),1)
        if img is not None:
            images.append(img)
    return images

# function to convert cv2-loaded image into RGB
def convertToRGB(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# scale image down or up while keeping original aspect ratio
def scale_image(image, ratio):
    scale_percent = ratio # percent of original size -> change to specify new size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    return cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

# returns all faces detected in input image using a DNN
def detectFaceOpenCVDnn(net, image):
    frameHeight = image.shape[0]
    frameWidth = image.shape[1]
    
    data = cv2.dnn.blobFromImage(image, 1.0, (300, 300), [104, 117, 123], False, False)
    
    net.setInput(data)
    detections = net.forward()
    bounding_boxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        # print(confidence)
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            if not x1 < 0 and not x2 > frameWidth and not y1 < 0 and not y2 > frameHeight:
                bounding_boxes.append([x1, y1, x2, y2])
                # cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight/150)), 8)
    return bounding_boxes

### Preprocessing of input images 
    1. Uses pretrained Caffe or TensorFlow model to perform face detection
    2. Crops training photos around detected faces
    3. Save cropped photos in new training set with same structure for loading into DataBunch

In [ ]:
# optional haar model to use to locate faces
# haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# will only choose partitions of the image that the DNN says minimum conf_threshold % probability of being a face
conf_threshold = 0.98

# directory containing images of people to train on. NOTE: must be in same directory as this notebook
raw_training_data_dir = 'intern_images'

#load DNN models
DNN = "CAFFE"
if DNN == "CAFFE":
    modelFile = "models/weights.caffemodel"
    configFile = "models/deploy.prototxt"
    net = cv2.dnn.readNetFromCaffe(configFile, modelFile)
else:
    modelFile = "opencv_face_detector_uint8.pb"
    configFile = "opencv_face_detector.pbtxt"
    net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)

# make directory for training images, set readable
if not os.path.exists('training_set'):
    os.mkdir('training_set', 0o777)
elif os.path.isdir('training_set'): 
    shutil.rmtree('training_set')
    os.mkdir('training_set', 0o777)

# load all input images, process and output cropped faces to new test directory
for dirname, dirnames, filenames in os.walk('./' + raw_training_data_dir):
    for subdirname in dirnames:
        
        # create subpath for specific class images for after cropping, and delete existing images if they exist
        sub_path = 'training_set/' + subdirname
        if not os.path.exists(sub_path):
            os.mkdir(sub_path, 0o777)
        elif os.path.isdir(sub_path): 
            shutil.rmtree(sub_path)
            os.mkdir(sub_path, 0o777)
            
        
        #print(os.path.join(dirname, subdirname))
        imgs = load_images_from_folder(os.path.join(dirname, subdirname))
        # print('loaded images for '+ subdirname)
        index = 0
        for img in imgs:
            # uncomment to 
            # faces = haar_cascade.detectMultiScale(img, scaleFactor = 1.2, minNeighbors = 4, minSize=(500, 500))
            
            # scale image down and detect faces using the DNN
            img = scale_image(img, 18)
            faces = detectFaceOpenCVDnn(net,img)
            # if there is one face in the frame, crop the frame to isolate the face
            if len(faces) >= 1:
                index+=1
                (x1,y1,x2,y2) = faces[0]
                cropped = img[y1:y2, x1:x2].copy()
                # uncomment to see all loaded pictures
                plt.figure()
                plt.imshow(convertToRGB(cropped))
                
                # create and store cropped face images
                filename = subdirname + "_" + '{0:04d}'.format(index)+".jpg"
                cv2.imwrite(os.path.join(sub_path , filename), cropped)
        print("loaded " + str(len(imgs)) + " images of " + subdirname)

### Load cropped test faces into ImageDataBunch and apply transforms
    Transforms include jitter, brightness adjustment, and contrast adjustment

In [ ]:
## applying transforms
tfms_list = [jitter(magnitude=(random.randrange(-3,3)/100), p=0.25), 
             contrast(scale=(0.5, 2.), p=0.5), brightness(change=(0.1, 0.9), p=0.5)]
#tfms_list = [rotate(degrees=270, p=1)]
tfms = [tfms_list, tfms_list]

## Declaring path of dataset
path_img = Path('/root/model_training/training_set')

## Loading data 
data = ImageDataBunch.from_folder(path=path_img, train='/', valid_pct=0.07, ds_tfms=tfms, bs=bs, size=(150,150))

## Normalizing data based on ImageNet parameters
data.normalize(imagenet_stats)

## Showing some contents of databunch to confirm it is loaded correctly with transforms
data.show_batch(rows=2)
print(data.classes)
len(data.classes),data.c

### Transfer learning on pretrained facenet_pytorch model

In [ ]:
def get_model(pretrained=True):
    if pretrained:
        facenet_base = InceptionResnetV1(pretrained='vggface2').eval()
    else:
        facenet_base = models.resnet18
    return facenet_base

## To create a model with pretrained weights
learn = cnn_learner(data, get_model, metrics=accuracy)
# learn.freeze_to(300)
learn.fit_one_cycle(10,1e-2)
learn.save("face_detection_" + str(datetime.utcnow().strftime("%m%d%Y")))

In [ ]:
print('Training results:  ' + str(learn.validate(learn.data.train_dl)))

In [ ]:
print('Validation results: ' + str(learn.validate(learn.data.valid_dl)))

In [ ]:
print(learn.loss_func)
preds,y,losses = learn.get_preds(with_loss=True)
interp = ClassificationInterpretation(learn, preds, y, losses)
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(9, figsize=(10,10))

In [ ]:
interp.plot_confusion_matrix(figsize=(5,5), dpi=60)

In [ ]:
learn.lr_find()
learn.recorder.plot()
plt.plot(learn.recorder.lrs[10:-5],learn.recorder.losses[10:-5],'r.')
plt.xscale('log')

In [ ]:
# Load pretrained model weights
for file in os.listdir("/root/model_training/training_set/models"):
    if file.endswith(".pth"):
        model_load = os.path.join("/mydir", file)
batch_size = 16    # just a random number

# Initialize model with the pretrained weights
map_location = lambda storage, loc: storage
if torch.cuda.is_available():
    map_location = None
# learn.load_state_dict(model_zoo.load_url(model_load, map_location=map_location))

torch_model = learn.model

# set the train mode to false since we will only run the forward pass.
torch_model.train(False)

# Input to the model
x = torch.randn(10, 3, 224, 224, device='cuda',requires_grad=True)

view_output=False

# Export the model
torch_out = torch.onnx._export(torch_model,             # model being run
                               x,                       # model input (or a tuple for multiple inputs)
                               "models/face_detector.onnx",    # where to save the model (can be a file or file-like object)
                               export_params=True,      # store the trained parameter weights inside the model file
                               verbose=view_output)     # whether or not to view onncxoutput progress
os.chmod("models/face_detector.onnx", 0o777)
print("loaded model in onnx")

In [ ]:
# Only run if you want to use caffe2 model in your application

os.system("pip install onnx")
import onnx
import caffe2.python.onnx.backend as backend

# Load the ONNX ModelProto object. model is a standard Python protobuf object
model = onnx.load("face_detector.onnx")

# Check that the IR is well formed
onnx.checker.check_model(model)

rep = backend.prepare(model, device="CUDA:0")

outputs = rep.run(np.random.randn(10, 3, 224, 224).astype(np.float32))

os.system("python convertCaffe.py ./models/face_detector.onnx + ./models/face_detector_"
        +str(datetime.utcnow().strftime("%m%d%Y"))+".prototxt ./models/face_detector_"
        +str(datetime.utcnow().strftime("%m%d%Y"))+".caffemodel")